In [ ]:
#Questions:

In [1]:
#importing packages
import pandas as pd
import os
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
import random
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score


In [2]:
# Trying to set the seed
np.random.seed(0)
random.seed(0)

In [3]:
#importing data set(s)
filename = 'default of credit card clients.xls'
nanDict = {} #this does nothing with this data set
df = pd.read_excel(filename,header=1,skiprows=0,index_col=0,na_values=nanDict)
df.rename(index=str, columns={"default payment next month": "defaultPaymentNextMonth"}, inplace=True)
df.dropna()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,defaultPaymentNextMonth
ID,,,,,,,,,,,,,,,,,,,,,
1,20000,2,2,1,24,2,2,-1,-1,-2,...,0,0,0,0,689,0,0,0,0,1
2,120000,2,2,2,26,-1,2,0,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
3,90000,2,2,2,34,0,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
4,50000,2,2,1,37,0,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
5,50000,1,2,1,57,-1,0,-1,0,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0
6,50000,1,1,2,37,0,0,0,0,0,...,19394,19619,20024,2500,1815,657,1000,1000,800,0
7,500000,1,1,2,29,0,0,0,0,0,...,542653,483003,473944,55000,40000,38000,20239,13750,13770,0
8,100000,2,2,2,23,0,-1,-1,0,0,...,221,-159,567,380,601,0,581,1687,1542,0
9,140000,2,3,1,28,0,0,2,0,0,...,12211,11793,3719,3329,0,432,1000,1000,1000,0


In [4]:
# Features and targets 
X = df.loc[:, df.columns != 'defaultPaymentNextMonth'].values
y = df.loc[:, df.columns == 'defaultPaymentNextMonth'].values
# above makes all columns except the last 
# into features, and the last is targets
# trainingShare = 0.5 

# XTrain, XTest, yTrain, yTest=train_test_split(X, y, train_size=trainingShare, \
#                                               test_size = 1-trainingShare,
#                                              random_state=seed)

# sc = StandardScaler()
# #removes mean, scales by std
# XTraint = sc.fit_transform(XTrain)
# XTestt = sc.transform(XTest)

# print(pd.DataFrame(XTraint))

In [10]:
# Categorical variables to one-hot's
onehotencoder = OneHotEncoder(categories="auto")
# sets number of elements in onehot vectors automatically, from data.
Xt= ColumnTransformer(
    [("", onehotencoder, [1,2,3]),],
    remainder="passthrough"
).fit_transform(X)
print(np.shape(Xt))

(30000, 33)


In [11]:
# Train-test split
trainingShare = 0.5 
seed  = 1
XTrain, XTest, yTrain, yTest=train_test_split(Xt, y, train_size=trainingShare, \
                                              test_size = 1-trainingShare,
                                             random_state=seed)

In [13]:
# Input Scaling
sc = StandardScaler()

# Remove one-hots
X_scale = np.c_[XTrain[0],XTrain[14:]]
#removes mean, scales by std
XTraint = sc.fit_transform([XTrain[0],XTrain[14:]])
XTestt = sc.transform(XTest)
print(XTraint[:5])

ValueError: all the input array dimensions except for the concatenation axis must match exactly

In [ ]:
# One-hot's of the target vector
Y_train_onehot, Y_test_onehot = onehotencoder.fit_transform(yTrain), onehotencoder.fit_transform(yTest)
#takes an array like [0,0,1,0,1,1,0]
#turns into a scipy compressed sparse row matrix, where a sparse matrix is stored by
# three vectors A, IA and JA
# A = all non-zero values, row major order
# IA = cumulative number of nonzero elements per row, IA[0]=0 by definition,
# JA =  the column index of each element of A
# See full matrix shape by Y_train_onehot.todense()

In [ ]:
# Remove instances with zeros only for past bill statements or paid amounts
'''
df = df.drop(df[(df.BILL_AMT1 == 0) &
                (df.BILL_AMT2 == 0) &
                (df.BILL_AMT3 == 0) &
                (df.BILL_AMT4 == 0) &
                (df.BILL_AMT5 == 0) &
                (df.BILL_AMT6 == 0) &
                (df.PAY_AMT1 == 0) &
                (df.PAY_AMT2 == 0) &
                (df.PAY_AMT3 == 0) &
                (df.PAY_AMT4 == 0) &
                (df.PAY_AMT5 == 0) &
                (df.PAY_AMT6 == 0)].index)
'''
df = df.drop(df[(df.BILL_AMT1 == 0) &
                (df.BILL_AMT2 == 0) &
                (df.BILL_AMT3 == 0) &
                (df.BILL_AMT4 == 0) &
                (df.BILL_AMT5 == 0) &
                (df.BILL_AMT6 == 0)].index)

df = df.drop(df[(df.PAY_AMT1 == 0) &
                (df.PAY_AMT2 == 0) &
                (df.PAY_AMT3 == 0) &
                (df.PAY_AMT4 == 0) &
                (df.PAY_AMT5 == 0) &
                (df.PAY_AMT6 == 0)].index)

#This seems to have dropped nobody in this data set

In [ ]:
lambdas=np.logspace(-5,7,13)
parameters = [{'C': 1./lambdas, "solver":["lbfgs"]}]#*len(parameters)}]
scoring = ['accuracy', 'roc_auc']
logReg = LogisticRegression(max_iter=1e6)
gridSearch = GridSearchCV(logReg, parameters, cv=5, scoring=scoring, refit='roc_auc')
#roc_auc and accuracy (and confusion_matrix) not imported, yet seems to work?

In [ ]:
# "refit" gives the metric used deciding best model. 
# See more http://scikit-learn.org/stable/auto_examples/model_selection/plot_multi_metric_evaluation.html
gridSearch.fit(XTrain, yTrain.ravel())
;

In [ ]:
def gridSearchSummary(method, scoring):
    """Prints best parameters from Grid search
    and AUC with standard deviation for all 
    parameter combos """
    
    method = eval(method)
    if scoring == 'accuracy':
        mean = 'mean_test_accuracy'
        sd = 'std_test_accuracy'
    elif scoring == 'auc':
        mean = 'mean_test_roc_auc'
        sd = 'std_test_roc_auc'
    print("Best: %f using %s" % (method.best_score_, method.best_params_))
    means = method.cv_results_[mean]
    stds = method.cv_results_[sd]
    params = method.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
gridSearchSummary('gridSearch','accuracy')
gridSearchSummary('gridSearch','auc')

In [ ]:
def createConfusionMatrix(method, printOut=True):
    """
    Computes and prints confusion matrices, accuracy scores,
    and AUC for test and training sets 
    """
    confusionArray = np.zeros(6, dtype=object)
    method = eval(method)
    
    # Train
    yPredTrain = method.predict(XTrain)
    yPredTrain = (yPredTrain > 0.5)
    cm = confusion_matrix(
        yTrain, yPredTrain) 
    cm = np.around(cm/cm.sum(axis=1)[:,None], 2)
    confusionArray[0] = cm
    
    accScore = accuracy_score(yTrain, yPredTrain)
    confusionArray[1] = accScore
    
    AUC = roc_auc_score(yTrain, yPredTrain)
    confusionArray[2] = AUC
    
    if printOut:
        print('\n###################  Training  ###############')
        print('\nTraining Confusion matrix: \n', cm)
        print('\nTraining Accuracy score: \n', accScore)
        print('\nTrain AUC: \n', AUC)
    
    # Test
    yPred = method.predict(XTest)
    yPred = (yPred > 0.5)
    cm = confusion_matrix(
        yTest, yPred) 
    cm = np.around(cm/cm.sum(axis=1)[:,None], 2)
    confusionArray[3] = cm
    
    accScore = accuracy_score(yTest, yPred)
    confusionArray[4] = accScore
    
    AUC = roc_auc_score(yTest, yPred)
    confusionArray[5] = AUC
    
    if printOut:
        print('\n###################  Testing  ###############')
        print('\nTest Confusion matrix: \n', cm)
        print('\nTest Accuracy score: \n', accScore)
        print('\nTestAUC: \n', AUC)    
    
    return confusionArray

In [ ]:
gulp = createConfusionMatrix('gridSearch')

In [ ]:
import matplotlib.pyplot as plt
import seaborn
import scikitplot as skplt

import warnings
import matplotlib.cbook
warnings.filterwarnings("ignore",category=matplotlib.cbook.mplDeprecation)

seaborn.set(style="white", context="notebook", font_scale=1.5, 
            rc={"axes.grid": True, "legend.frameon": False,
"lines.markeredgewidth": 1.4, "lines.markersize": 10})
seaborn.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 4.5})

yPred = gridSearch.predict_proba(XTest) 
print(yTest.ravel().shape, yPred.shape)

#skplt.metrics.plot_cumulative_gain(yTest.ravel(), yPred_onehot)
skplt.metrics.plot_cumulative_gain(yTest.ravel(), yPred)

defaults = sum(yTest == 1)
total = len(yTest)
defaultRate = defaults/total
def bestCurve(defaults, total, defaultRate):
    x = np.linspace(0, 1, total)
    
    y1 = np.linspace(0, 1, defaults)
    y2 = np.ones(total-defaults)
    y3 = np.concatenate([y1,y2])
    return x, y3

x, best = bestCurve(defaults=defaults, total=total, defaultRate=defaultRate)    
plt.plot(x, best)    


plt.show()

# b starts here i guess

In [ ]:
"""
Takes two arrays of equal length and calculates the "Accuracy score"
"""
def Accuracy(t,y):
    n = len(t)
    m = len(y)
    if m!=n:
        print("Vectors passed to the Accuracy function is of different lengths.")
        return 0
    s = [1 if i else 0 for i in np.equal(t,y)]
    return np.sum(s)/n

In [ ]:
a=[1,2,3]
b=[1,7,3,9]
zz = Accuracy(a,b)
print(zz)
a=[1,2,3]
b=[1,7,3]
zz = Accuracy(a,b)
print(zz)

In [ ]:
# set initial stuff
N_epochs = 10
N_batches = 5
validation_portion=0.1
theta = np.random.randn(Xt.shape[1]-1,1)
eta = 0.1
# Create shuffled dataset
df.sample(frac=1)
print(np.shape(Xt))
#Split into train and validation
Xtrain,Xtest,ytrain,ytest = train_test_split(Xt[:,:-1],Xt[:,-1],test_size=validation_portion)
# Xtrain,Xtest,ytrain,ytest = train_test_split(df.drop(['defaultPaymentNextMonth'],axis=1),df['defaultPaymentNextMonth'],test_size=validation_portion)

# print(np.shape(Xtrain))
# Split data into minibatches
batch_len = int(Xtrain.shape[1]/N_batches)+1
batches=[]
batch_targets=[]
for i in range(N_batches):
    batches.append(Xtrain[i*batch_len:(i+1)*batch_len][:])
    batch_targets.append(ytrain[i*batch_len:(i+1)*batch_len][:])

# do all the stuff!
bipp = []
for j in range(1,N_epochs+1):
    for i in range(N_batches):
        for h in range((batches[i]).shape[0]):
            bat = batches[i][h].toarray()
            tar = batch_targets[i][h].toarray()
            gradients = 2.0*bat.T@((bat@theta)-tar)
#             print(gradients)
            bipp.append(gradients)
#             theta -= eta*gradients
#             print(eta*gradients)
#     #         z=
#             print(np.shape(gradients))
#             print(np.shape(gradients))
            a=0
# print(theta)

In [ ]:
print(np.max(bipp))

In [ ]:
from sklearn.linear_model import SGDRegressor
import numpy
sgdreg = SGDRegressor(tol=1e-3,max_iter = 500, penalty=None, eta0=0.1)
sgdreg.fit(X,y.ravel())
print("sgdreg from scikit")
print(sgdreg.intercept_,sgdreg.coef_)

In [ ]:
no scaling or onehotting performed...